<a href="https://colab.research.google.com/github/vbwanere/SEDS_in_jax/blob/main/gmm/GMM_from_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Project Title**

Project desciption here...

# 1\. Section 1:

"""
sample markdown
"""

## 1.1 Subsection 1:

In [ ]:
# sample code here

sample figure here

## 1.2 Subsection 2:

In [ ]:
"""
sample code here
"""

### 1.2.1 item 1:

In [ ]:
"""
sample code here
"""

### 1.2.2 item 2:

In [ ]:
"""
sample code here
"""

# 2\. Section 2:

## 2.1\. Subsection 1:
### Subsection description in one line: